In [1]:
import torch
from train import train, init_llama3,sys_prompt
from dataset import GPTDataSet
from datasets import load_dataset
from trl import DataCollatorForCompletionOnlyLM
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained("t-tech/T-lite-it-1.0")

model, tokenizer = init_llama3()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
import pandas as pd

print(torch.cuda.is_available())
l = pd.read_csv("category_triplets.csv")
l.head()

True


,user_category_1,user_category_2,predicted_category
0,Комплектующие для ПК,Музыка,Антикварные книги
1,Детям и родителям,Личная гигиена,Антикварные книги
2,Автомобили новые,"Буклеты, листовки и брошюры",Антикварные книги
3,Музыка,Антикварные книги,Ароматерапия
4,Уход за волосами,Компьютерная и офисная мебель,Ароматерапия


In [3]:

dataset = load_dataset("json",data_files= {"train":["train.jsonl"], "test":["test.jsonl"]})

collator = None
dataset

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 11057
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 2765
    })
})

In [4]:
train(model,tokenizer, dataset, collator = collator,epoches_count=2)

/home/student/bondarev/venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_num_proc. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/student/bondarev/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/student/bondarev/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:314: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/student/bondarev/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You pas

trainable params: 6,881,280 || all params: 7,619,637,760 || trainable%: 0.0903


Step,Training Loss
1,2.458000
2,2.527300
3,2.472900
4,2.463500
5,2.374100
6,2.160600
7,1.934200
8,1.764000
9,1.648300
10,1.504000


TrainOutput(global_step=750, training_loss=0.11235647667447726, metrics={'train_runtime': 629.8095, 'train_samples_per_second': 9.527, 'train_steps_per_second': 1.191, 'total_flos': 8.58616609277952e+16, 'train_loss': 0.11235647667447726, 'epoch': 0.5425935973955507})

In [5]:
test_prompt = "Определи, используя знания о человеческой психологии, из какой одной категории товаров, существующих в \
нашем интернет магазине, пользователь купит следующий товар, если он уже купил: {}, {}".format("Туристические наборы","Компьютерная техника")

messages = [
    {"role": "system", "content": "Ты T-lite, виртуальный ассистент в Т-Технологии. Твоя задача - быть полезным диалоговым ассистентом."},
    {"role": "user", "content": test_prompt}
]
test_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
tokens = tokenizer(test_prompt,return_tensors="pt")
print(tokens)

model.eval()

with torch.no_grad():
    result = model.generate(input_ids=tokens["input_ids"].cuda(), max_new_tokens=100,generation_config=None)
    print(result)
output = tokenizer.decode(result[0][-230:],skip_special_tokens=True)
print(output)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'input_ids': tensor([[151644,   8948,    198,  33995,   4552,    350,  78482,     11,   5805,
          77646,  88248, 130451,  20396, 128698,  34011,  18673,   5805,  50318,
             12,  33995,  97185, 127812,  46173,     13,  50318,   5474,  60290,
         133771,   1478,    481,  61631,  28519,  31885, 125439, 143755,  15952,
          14497, 134163,  20396, 128698,  34011,  18673,  12228,     13, 151645,
            198, 151644,    872,    198,  20353,   8005,  42975,  60542,     11,
          89695,   3780,   4235, 143198,  38180,   8215, 126634,  55757,  22484,
         128588, 140582, 138445,     11,  23064, 134207, 133304, 143118, 140659,
             11, 127339,  71019,  66645,   5805, 135999, 130231,  11310,  33393,
          18671,  18943,   1504,     11,  45295,  25428,   7665,  48807,   9358,
          92029,  71019,  43838,  73712,     11,  46253,  98745,  90777,   7665,
          48807,  63192,     25,  50318,   3780, 127627, 130102, 140202,   9062,
           455

<h1>Matching</h1>

In [6]:
from difflib import SequenceMatcher
import json
from tqdm import tqdm
categories = ['Игровые жанры', 'PlayStation', 'Nintendo', 'Xbox', 'Игровые приставки', 'PC', 'Mobile gaming', 'Ретро-консоли', 'Игровые картриджи', 'Игровая атрибутика', 'Игровые наушники', 'Геймпады', 'Очки виртуальной реальности', 'Рули и педали', 'Аксессуары для игровых приставок', 'Аксессуары для киберспорта', 'Аккумуляторы и зарядки', 'Сумки и чехлы', 'Запчасти для игровых консолей', 'Телефоны и смарт-часы', 'Наушники и аудиотехника', 'Ноутбуки, планшеты и электронные книги', 'Телевизоры и видеотехника', 'Комплектующие для ПК', 'Компьютеры и периферия', 'Игровые приставки и ноутбуки', 'Офисная техника', 'Фото и видеокамеры', 'Охранные системы и видеонаблюдение', 'Умный дом', 'Квадрокоптеры и аксессуары', 'Навигаторы', 'Оптические приборы', 'Часы и электронные будильники', 'Аксессуары для электроники', 'Садовая мебель', 'Компьютерная и офисная мебель', 'Мягкая мебель', 'Столы и стулья', 'Мебель для хранения', 'Мебель для спальни и комплектующие', 'Мебель для кухни', 'Мебель для ванной', 'Бескаркасная мебель', 'Детская мебель', 'Мебель для бизнеса', 'Сейфы и архивные шкафы', 'Музыка', 'Виниловые пластинки', 'Видео', 'Автомобили новые', 'Товары для школы и обучения', 'Игрушки и игры', 'Спорт и игры на улице', 'Подгузники и гигиена', 'Детская комната', 'Коляски и автокресла', 'Детское питание', 'Товары для кормления', 'Товары для мам', 'Письменные принадлежности', 'Бумага', 'Бумажная продукция', 'Папки и файлы', 'Обложки', 'Демонстрационные доски', 'Офисные принадлежности', 'Подставки и визитницы', 'Печати и штампы', 'Калькуляторы', 'Картриджи для лазерных принтеров', 'Картриджи для струйных принтеров', 'Чертежные принадлежности', 'Оборудование и принадлежности для торговли', 'Полиграфическое оборудование', 'Сублимационная печать', 'Посуда и кухонные принадлежности', 'Текстиль', 'Освещение', 'Декор и интерьер', 'Дача и сад', 'Цветы, растения и горшки', 'Хозяйственные товары', 'Хранение вещей', 'Товары для бань и саун', 'Товары для праздников', 'Религия и эзотерика', 'Ритуальные товары', 'Женщинам', 'Мужчинам', 'Детям', 'Уход и аксессуары', 'Кольца', 'Серьги', 'Браслеты', 'Украшения на шею', 'Комплекты украшений', 'Шармы', 'Броши и значки', 'Пирсинг', 'Часы', 'Детские украшения', 'Запонки и зажимы', 'Сувениры', 'Шнурки', 'Аксессуары для волос', 'Религиозные украшения', 'Электронные сертификаты', 'Онлайн-подписки', 'Игры и игровые подписки', 'Программное обеспечение', 'Облачные хранилища', 'Услуги', 'Туризм и отдых на природе', 'Рыбалка', 'Охота и стрельба', 'Лодки и лодочные моторы', 'Одежда для рыбалки и охоты', 'Обувь для рыбалки и охоты', 'Тактическая одежда и аксессуары', 'Наборы для выживания', 'Нехудожественная литература', 'Бизнес-литература', 'Художественная литература', 'Детям и родителям', 'Учебная литература', 'Электронные книги', 'Букинистика', 'Комиксы', 'Манга', 'Печать по требованию', 'Подарочные издания книг', 'Журналы и газеты', 'Литература на иностранных языках', 'Электронные аудиокниги', 'Подписки на электронные книги и аудиокниги', 'Антикварные книги', 'Книги б/у', 'Блокноты и ежедневники', 'Буклеты, листовки и брошюры', 'Путешествия', 'Уход за волосами', 'Уход за лицом', 'Уход за телом', 'Макияж', 'Маникюр и педикюр', 'Парфюмерия', 'Загар и защита от солнца', 'Мужская косметика', 'Детская косметика и парфюмерия', 'Беременным и кормящим', 'Аппаратная косметология и массаж', 'Ароматерапия', 'Оборудование и материалы для тату-салона', 'Мебель и оборудование для салонов красоты', 'Лекарственные средства', 'Витамины и БАДы', 'Парафармацевтика', 'Оптика', 'Медицинские приборы', 'Медицинское оборудование', 'Аксессуары и комплектующие', 'Медицинские изделия и расходные материалы', 'Одежда и текстиль медицинские', 'Медицинские инструменты', 'Материалы и средства стоматологические', 'Товары для гигиены', 'Товары для реабилитации', 'Ортопедия', 'Ручные массажеры и аппликаторы', 'Мебель медицинская', 'Униформа и рабочая одежда', 'Уход за одеждой', 'Климатическая техника', 'Техника для кухни', 'Техника для дома', 'Техника для красоты и здоровья', 'Крупная бытовая техника', 'Техника для общепита', 'Коллекционирование', 'Посуда и утварь', 'Предметы военной истории', 'Предметы интерьера', 'Предметы быта', 'Мебель', 'Украшения', 'Часы наручные', 'Канцелярия', 'Приборы и инструменты', 'Живопись и графика', 'Галантерея', 'Товары для курения', 'Музыкальные инструменты', 'Религиозная атрибутика', 'Запчасти для легковых автомобилей', 'Автоаксессуары и принадлежности', 'Масла и автохимия', 'Шины и диски', 'Автозвук', 'Транспортные средства', 'Уход за автомобилем', 'Электроника для автомобиля', 'Инструменты и оборудование', 'Гараж и автосервис', 'Аккумуляторы и аксессуары', 'Мототовары', 'Запчасти для грузовиков и спецтехники', 'Автолитература', 'Для кошек', 'Для собак', 'Для грызунов и хорьков', 'Для птиц', 'Для рыб и рептилий', 'Для лошадей', 'Ветаптека', 'Фермерское хозяйство', 'Ветеринарное оборудование', 'Бытовая химия', 'Личная гигиена']

def match(sample):
    if sample in categories:
        return sample
    for i in categories:
        if SequenceMatcher(None, i, sample).ratio() > 0.7:
            return i
    return "Неизвестно"

In [7]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from train import init_llama3, sys_prompt

def evaluate(model,tokenizer,cat1,cat2):
    batch_size = len(cat1)
    messages = []

    for i in range(batch_size):
        test_prompt = "Какие три следующие категории понравятся пользователю, если до этого ему понравились следующие категории: {}, {}".format(cat1[i],cat2[i])

        message = [
            {"role": "system", "content": """Ты T-lite, виртуальный ассистент в Т-Технологии. Твоя задача - рекомендовать пользователям следующие категории, которые могут им понравиться.
                    Пользователи проявляют интерес к игровым жанрам и сувенирам, что указывает на увлечение развлечениями. Высокая вероятность лайка для новых автомобилей свидетельствует об интересе к крупным и качественным покупкам, вызывающим вау-эффект. Они ценят комфорт, что отражается в интересе к мягкой мебели и текстилю. Забота о здоровье проявляется в интересе к витаминам и БАДам, а также технике для красоты. Пользователи активно интересуются электроникой, включая игровые приставки и мобильные игры, и аксессуары, такие как рули с педалямми.Также пользователям нравятся товары для дома и личной гигиены, что подчеркивает их стремление к уюту и комфорту."""},
            {"role": "user", "content": test_prompt}
        ]
            
        test_prompt = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=True
        )
        messages.append(test_prompt)
    
    tokens = tokenizer(messages,return_tensors="pt",padding=True,padding_side='left')
    with torch.no_grad():
        result = model.generate(input_ids=tokens["input_ids"].cuda(), max_new_tokens=100)
    output = tokenizer.batch_decode(result,skip_special_tokens=True)
    for i in range(batch_size):
        output[i] = output[i][output[i].index("assistant")+len("assistant"):].strip()
    return output

In [8]:
costile = ""
with open("test.jsonl", "r") as f:
    for item in f:
        l = json.loads(item)
        costile = costile + l["messages"][1]["content"]+"  \n"
#costile

In [9]:
#del evaluate
#from evaluate import evaluate
import itertools
import math
pairs = []
unknown = 0
total = 0
batch_size = 32


data_old = list(itertools.product(categories,categories))

data = []
for i,j in tqdm(data_old):
    if i+", "+j in costile:
        data.append([i,j])

print(len(data))
t = tqdm(range(math.ceil(len(data)/batch_size)))
for iters in t:
    i,j = zip(*data[iters*batch_size: (iters+1)*batch_size])
    resp = evaluate(model,tokenizer, i, j)
    for k in range(len(i)):
        resp[k] = resp[k].split("\n")
        #print(resp[k])
        for o in range(len(resp[k])):
            total += 1
            #resp[k][o] = match(resp[k][o])
            if resp[k][o] == "Неизвестно":
                unknown += 1
        #print(resp[k])
        #print()
        resp[k] = "; ".join(resp[k])
        pairs.append([i[k],j[k],resp[k]])
        
    t.set_postfix_str("Non-matched: {}, total: {}, \tcategory: {}".format(unknown,total,resp[0][:10] ))#+ ' '*(len(resp)-40))

with open("pairs750_test.json", 'w') as jsonl_file:
    json_obj = json.dump(pairs,jsonl_file,ensure_ascii=False,indent = 1)



00%|██████████| 47961/47961 [00:06<00:00, 7169.46it/s]

2837



00%|██████████| 89/89 [03:40<00:00,  2.47s/it, Non-matched: 0, total: 5673, 	category: Косметика ]

In [10]:
costile = ""
with open("train.jsonl", "r") as f:
    for item in f:
        l = json.loads(item)
        costile = costile + l["messages"][1]["content"]+"  \n"
#costile

import itertools
import math
pairs = []
unknown = 0
total = 0
batch_size = 32


data_old = list(itertools.product(categories,categories))

data = []
for i,j in tqdm(data_old):
    if i+", "+j in costile:
        data.append([i,j])

print(len(data))
t = tqdm(range(math.ceil(len(data)/batch_size)))
for iters in t:
    i,j = zip(*data[iters*batch_size: (iters+1)*batch_size])
    resp = evaluate(model,tokenizer, i, j)
    for k in range(len(i)):
        resp[k] = resp[k].split("\n")
        #print(resp[k])
        for o in range(len(resp[k])):
            total += 1
            #resp[k][o] = match(resp[k][o])
            if resp[k][o] == "Неизвестно":
                unknown += 1
        #print(resp[k])
        #print()
        resp[k] = "; ".join(resp[k])
        pairs.append([i[k],j[k],resp[k]])
        
    t.set_postfix_str("Non-matched: {}, total: {}, \tcategory: {}".format(unknown,total,resp[0][:10] ))#+ ' '*(len(resp)-40))

with open("pairs750_train.json", 'w') as jsonl_file:
    json_obj = json.dump(pairs,jsonl_file,ensure_ascii=False,indent = 1)


00%|██████████| 47961/47961 [00:23<00:00, 2025.58it/s]

10534



00%|██████████| 330/330 [13:23<00:00,  2.44s/it, Non-matched: 0, total: 20987, 	category: Mobile gam]